# Import Packages

In [1]:
# Cell 1 — Imports

import pandas as pd
import numpy as np
import plotly.io as pio

# Ensure plotly renders inline
pio.renderers.default = "notebook_connected"

# Import your tsforge modules
from tsforge.plots.charts.plot_timeseries import plot_timeseries


## Build sample dataset

In [2]:
# FAKE DATA 

import numpy as np
import pandas as pd

# For reproducibility
rng = np.random.default_rng(42)

# -----------------------------
# ACTUALS: df_actuals
# -----------------------------
dates = pd.date_range("2020-01-01", periods=90, freq="D")
ids = ["FOODS_1", "BEV_2", "DAIRY_3"]

rows = []
for uid in ids:
    base_level = {
        "FOODS_1": 50,
        "BEV_2": 80,
        "DAIRY_3": 40,
    }[uid]

    seasonal = np.sin(np.linspace(0, 3 * np.pi, len(dates))) * 10
    noise = rng.normal(scale=5, size=len(dates))
    trend = np.linspace(0, 15, len(dates))

    if uid == "DAIRY_3":
        y = base_level + seasonal + noise              # mostly flat-ish
    else:
        y = base_level + seasonal + noise + trend      # trending

    for d, v in zip(dates, y):
        rows.append({
            "unique_id": uid,
            "ds": d,
            "y": float(max(v, 0.0)),   # no negatives
        })

df_actuals = pd.DataFrame(rows)

# -----------------------------
# FORECAST: df_fcst (single model + 80/95 PIs)
# -----------------------------
h = 14
fc_dates = pd.date_range(df_actuals["ds"].max() + pd.Timedelta(days=1),
                         periods=h, freq="D")

rows = []
for uid in ids:
    sub = df_actuals[df_actuals["unique_id"] == uid]
    base_last = sub["y"].iloc[-1]

    seasonal = np.sin(np.linspace(0, 2 * np.pi, h)) * 8
    noise = rng.normal(scale=3, size=h)
    trend = np.linspace(0, 5, h)

    mean = base_last + seasonal + trend
    yhat = mean + noise

    lo80 = yhat - 5
    hi80 = yhat + 5
    lo95 = yhat - 8
    hi95 = yhat + 8

    for d, yh, lo8, hi8, lo9, hi9 in zip(fc_dates, yhat, lo80, hi80, lo95, hi95):
        rows.append({
            "unique_id": uid,
            "ds": d,
            "yhat": float(yh),
            "yhat-lo-80": float(lo8),
            "yhat-hi-80": float(hi8),
            "yhat-lo-95": float(lo9),
            "yhat-hi-95": float(hi9),
        })

df_fcst = pd.DataFrame(rows)

# -----------------------------
# FORECAST: df_fcst_multi (two models)
# -----------------------------
rows = []
for uid in ids:
    base_fc = df_fcst[df_fcst["unique_id"] == uid].copy()

    for model_name, bias in [("Naive", 0.0), ("ML", 3.0)]:
        yhat = base_fc["yhat"] + bias

        for (_, r), yh in zip(base_fc.iterrows(), yhat):
            rows.append({
                "unique_id": uid,
                "ds": r["ds"],
                "model": model_name,
                "yhat": float(yh),
                "yhat-lo-80": float(yh - 5),
                "yhat-hi-80": float(yh + 5),
            })

df_fcst_multi = pd.DataFrame(rows)

# -----------------------------
# EVENTS: events_df
# -----------------------------
events_df = pd.DataFrame({
    "ds": pd.to_datetime([
        "2020-01-15",   # in-history promo
        "2020-02-14",   # Valentine's
        "2020-03-01",   # price change
        "2020-04-01",   # event during forecast horizon
    ]),
    "event": ["Promo A", "Valentine's", "Price Change", "Spring Reset"],
})

# -----------------------------
# ANOMALIES: anoms_df
# -----------------------------
anom_rows = []
for uid in ids:
    sub = df_actuals[df_actuals["unique_id"] == uid]
    # pick 2 random dates from the middle of history
    sample_dates = sub["ds"].iloc[20:70].sample(2, random_state=hash(uid) % 2**32)
    for d in sample_dates:
        anom_rows.append({"unique_id": uid, "ds": d})

anoms_df = pd.DataFrame(anom_rows)


In [3]:
events_df.head(2)

,ds,event
0,2020-01-15,Promo A
1,2020-02-14,Valentine's


## Test overlay

In [4]:
# Cell 3 — overlay test (FA theme)

fig = plot_timeseries(
    df=df_actuals,
    forecast=df_fcst,
    id_col="unique_id",
    date_col="ds",
    value_col="y",
    level=[80, 95],
    events_global=events_df,
    anomalies=anoms_df,
    mode="overlay",
    theme="fa",
)
fig


## Test Facet

In [5]:
# Cell 4 — facet mode test

fig = plot_timeseries(
    df_actuals,
    forecast=df_fcst,
    id_col="unique_id",
    date_col="ds",
    value_col="y",
    events_global=events_df,
    anomalies=anoms_df,
    level=[80, 95],
    mode="facet",
    theme="fa",
)
fig


## Test Dropdown

In [6]:
# Cell 5 — dropdown mode test

fig = plot_timeseries(
    df_actuals,
    forecast=df_fcst,
    id_col="unique_id",
    date_col="ds",
    value_col="y",
    level=[80, 95],
    events_global=events_df,
    anomalies=anoms_df,
    mode="dropdown",
    theme="fa",
)
fig


## Test Smoothing

In [7]:
# Cell 6 — smoothing test

fig = plot_timeseries(
    df_actuals,
    id_col="unique_id",
    date_col="ds",
    value_col="y",
    smooth_window=7,
    mode="overlay",
    theme="fa",
)
fig


## Test Grouping / Aggregation

In [8]:
# Cell 7 — Test group_col / aggregation

df_grouped = df_actuals.copy()
df_grouped["department"] = df_grouped["unique_id"].str.split("_").str[0]

fig = plot_timeseries(
    df_grouped,
    id_col="unique_id",
    date_col="ds",
    value_col="y",
    group_col="department",
    agg="mean",
    mode="overlay",
    theme="fa",
)
fig


## Test Themes

In [9]:
# Cell 8 — Test all themes

for t in ["fa", "mckinsey", "minimal", "dark"]:
    print("Theme:", t)
    fig = plot_timeseries(
        df_actuals,
        id_col="unique_id",
        date_col="ds",
        value_col="y",
        mode="overlay",
        theme=t,
    )
    fig.show()


Theme: fa


Theme: mckinsey


KeyError: 'axis_color'